In [3]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
print(text[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


## Tokenize Input Text
Translate individual characters to embeddings. Many different options for embedding characters/tokens - e.g., BPE (byte pair encoding) for encoding subword tokens

Generally, tradeoff between size of vocabulary and size of token embedding

In [6]:
# simple character to integer encoding
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # take string, return list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # take list of integers, output a string


In [7]:
# embed entire text and push into tensor object
import torch
data = torch.tensor(encode(text), dtype =torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [8]:
# split dataset into train/val split (90%/10%)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [9]:
# implement context length 
# for a data chunk of size n+1, there are n training examples
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When input is {context} the target: {target}")

When input is tensor([18]) the target: 47
When input is tensor([18, 47]) the target: 56
When input is tensor([18, 47, 56]) the target: 57
When input is tensor([18, 47, 56, 57]) the target: 58
When input is tensor([18, 47, 56, 57, 58]) the target: 1
When input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [12]:
# implement random minibatch sampling
torch.manual_seed(1337)
batch_size = 4 # number of batches to process in parallel
block_size = 8 # context length

def get_batch(split):
    data = train_data if split == 'train' else val_data
    # get batch_size number of starting values randomly from dataset
    # with at least block_size characters of runway
    ix = torch.randint(len(data) - block_size, (batch_size,))

    # stack the values so each sequence is a row vector
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print(xb.shape)
print(xb)
print(yb.shape)
print(yb)


torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [15]:
# pytorch bigram language model
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # inits simple embedding table
        # each token reads logits for next token 
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers
        # B = batch_size
        # T = block_size
        # C = vocab_size
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:

            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            # compute loss by comparing logit outputs to actual targets
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            
            # get values for last element in context length
            logits = logits[:, -1, :] # becomes (B, C) 
            
            # get prob distribution for each batch across vocab values
            probs = F.softmax(logits, dim=-1) # (B, C)

            # sample from that distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B,1)

            # append sampled index value to idx tensor
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size=vocab_size)
logits, loss = m(xb, yb)
# these are the logit prediction scores for each of the positions in each minibatch
print(logits.shape)
print(loss)




torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [16]:
start = torch.zeros((1,1), dtype=torch.long) # initialize start of sequence
print(decode(m.generate(start, max_new_tokens=100)[0].tolist()))


Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [17]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [22]:
# Conduct training
batch_size=32
for steps in range(10000):
    # sample data
    xb, yb = get_batch('train')

    # evaluate loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.465181589126587


In [23]:
start = torch.zeros((1,1), dtype=torch.long) # initialize start of sequence
print(decode(m.generate(start, max_new_tokens=500)[0].tolist()))


He g sounomy.

TICLKINourushes t bladoonju cie st t tree IIOLENREThand t, aghath,
MI: Fol'storevend os tho geat; tisheer lid al'st bal l are, tour bu olll pt my.
FR tr beser fare, wer

Whoto y oo nd plerlethor myseres: omathita illt w,
Taley myocomeat nethest,
NGLAnd,
Anen!
3QUMI; frthily, my Ise IIAn,
Five.
BOUCARI ke y vese o ye foy VENRYBireswo n

ABaXqusune y my n;
'e w: t w; MELULo INen meng te t:
in ticothingh, much s ESThondiryrs y ve:

The thafenda merar
WAng, aothace Pay, ty---
Tu whea 


In [35]:
# Self Attention 
torch.manual_seed(1337)
B,T,C = 4,8,2 # batch_size, context_length, vocab_size
x = torch.randn(B,T,C)
print(x.shape)

torch.Size([4, 8, 2])


In [36]:
# We want x[b,t] = mean(x[b,i]) for all i <=t
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] #(t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [46]:
# we can do this rolling average more efficiently using matrix multiplication

wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)

xbow2 = wei @ x # (B, T, T) @ (B, T, C) -> (B, T, C)

In [50]:
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros(T, T)
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim = -1)
print(wei)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])


In [47]:
# version 3: use softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros(T, T)
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim = -1)
xbow3 = wei @ x


In [31]:
# we can do this rolling average more efficiently using matrix multiplication
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a/torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print(a)
print(b)
print(c)

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])
